<a href="https://colab.research.google.com/github/ryderwishart/biblical-machine-learning/blob/main/qa_data_rationale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 73.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pi

In [2]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.2/834.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.6 MB/s eta 0:00:00


In [3]:
import torch
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

In [ ]:
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

In [ ]:
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt, verbose=True)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context, verbose=True)

In [ ]:
context = """George Washington (February 22, 1732[b] – December 14, 1799) was an American military officer, statesman,
and Founding Father who served as the first president of the United States from 1789 to 1797."""

instruction = "When was George Washington president?"

prediction = llm_context_chain.predict(instruction=instruction, context=context).lstrip()

print(prediction)

# Custom Template

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
examples = [
    
{"question": "What is the class of the word 'λέγω' and what does it mean?",
"answer": "λέγω is a verb meaning 'saying'.",
"rationale": ["The linguistic data states that the class of the word 'λέγω' is a verb ('V-PAP-NSM').",
"The gloss provided for 'λέγω' is 'saying', indicating its meaning.",
"The lemma form of the word is also mentioned as λέγω.",
"The lexical feature section confirms that λέγω is a verb with the lemma form λέγω.",
"Based on the data, the answer is correct."],
"score": 5},
{"question": "How is the word 'λέγω' parsed and what is its Strong's number?",
"answer": "λέγω is parsed as a V-PAP-NSM and has a Strong's number of 3004.",
"rationale": ["The linguistic data explicitly states that the word 'λέγω' is parsed as a V-PAP-NSM.",
"The data also mentions that the Strong's number for 'λέγω' is 3004.",
"The lexical feature section confirms the parsing information and the Strong's number.",
"Based on the data, the answer is correct."],
"score": 5},
{"question": "What is the semantic domain of 'λέγω'?",
"answer": "The semantic domain of λέγω is 33.68.",
"rationale": ["The provided linguistic data does not explicitly mention the semantic domain of 'λέγω'.",
"The cultural/encyclopedic knowledge section mentions the semantic domain entry in Louw and Nida's 'Greek-English Lexicon of the New Testament: Based on Semantic Domains' as 33.69, but it does not directly link it to 'λέγω'.",
"Since there is no direct evidence or connection between 'λέγω' and the specific semantic domain of 33.68, the answer cannot be justified based on the linguistic data provided."],
"score": 2}
]

In [ ]:
example_prompt = PromptTemplate(input_variables=['question', 'answer', 'rationale', 'score'],
                                template="Question: {question}\nAnswer: {answer}\nRationale: {rationale}\n{score}")

print(example_prompt.format(**examples[0]))

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix='Question: {input}', 
    input_variables=['input']
)

In [ ]:
context = """
## 1. Lexical features
- class: λέγω is a verb,
- gloss: meaning "saying."
- lemma: The lemma form of this word is λέγω,
- morph: and it is parsed as a V-PAP-NSM
- strong: with a Strong's number of 3004.
## 2. Syntactic context and function
- case: Grammatical case: nominative, genitive, dative, accusative, or vocative: nominative,
- gender: Grammatical gender values: masculine,
- mood: Grammatical mood: participle,
- number: Grammatical number: singular,
- tense: Grammatical tense form: present,
- voice: Grammatical voice: active,
- Semantic configuration: This word is the Event [A0: Ἰωάννης] [λέγω]  / [Source: John] [Process: saying] , and it has the following data: {'xml:id': 'n40003002001', 'ref': 'MAT 3:2!1', 'word': 'λέγων', 'lemma': 'λέγω', 'gloss': 'saying', 'frame': 'A0:n40003001007', 'frame_verb_id': 'n40003002001', 'frame_verb': 'λέγω', 'adjunct_label': 'Relation', 'adjunct_node_head': 'n40003002001', 'semantic_role_label': 'Event', 'semantic_role_node_head': 'n40003002001'}
## 4. Social context
This word is part of the passage 'The Ministry of John the Baptist'
Which begins as a addressee-more-passive unclear conceptual-ie-internally-oriented opposing phonic spoken instructing monological distant ancillary interpersonal on-someones-side situation
Addressee-more-passive process sharing refers to situations where the recipient of the message is less actively involved in the communication process, such as listening or reading without providing feedback.
Unclear control refers to situations where it is not evident who holds authority or power.
Conceptual field values involve abstract or theoretical activities, such as theological or philosophical discussions.
Opposing value orientation predisposition refers to participants who hold different views or are antagonistic toward each other's positions.
Phonic channel refers to communication through sound, such as spoken language or music.
Spoken medium refers to communication that takes place through speech, either in face-to-face conversations or through audio recordings.
Instructing goals are centered around teaching, explaining, or providing guidance to others.
Monological refers to situations with only one speaking participant, such as a monologue or a soliloquy.
Distant social distance refers to situations where participants have a distant relationship or are not familiar with each other.
Ancillary language role refers to situations where language plays a secondary or supporting role in carrying out the activity or achieving the goal.
Interpersonal activity focus pertains to the social interaction between participants, focusing on their roles, relationships, and attitudes.
On-someones-side publicity refers to a situation where participants actively support a particular stance or outcome.

And ends as a addressee-more-passive conceptual-ie-internally-oriented institutional opposing projecting spoken phonic monological distant constitutive interpersonal on-someones-side situation
Institutional control refers to situations where one participant or a group of participants hold authority or power over others.
Projecting goals involve making predictions, prophesying, or discussing future events or possibilities.
Spoken medium refers to communication that takes place through speech, either in face-to-face conversations or through audio recordings.
Constitutive language role refers to situations where language is the primary means of carrying out the activity or achieving the goal.
Interpersonal activity focus pertains to the social interaction between participants, focusing on their roles, relationships, and attitudes.
On-someones-side publicity refers to a situation where participants actively support a particular stance or outcome.
During the passage, the situation:
gained the following features: projecting, constitutive, institutional
lost the following features: ancillary, instructing, unclear
## 5. Cultural/encyclopedic knowledge
- domain: Semantic domain information from the Semantic Dictionary of Biblical Greek (SDBG): Speak, Talk,
- ln: The semantic domain entry in Louw and Nida's, 'Greek-English Lexicon of the New Testament: Based on Semantic Domains'.: 33.69,
"""

prediction = llm_context_chain.predict(instruction=prompt.format(input='What are the different values for grammatical case and gender?'), context=context).lstrip()

print(prediction)